In [2]:
import sys
import os
path = os.path.abspath('')
path = os.path.dirname(path)
sys.path.insert(1, str(path))


In [3]:

from datetime import date
import os
import load_csv


data_frame = load_csv.load(path + "/data/clean_data.csv")
context = load_csv.getContext()

your 131072x1 screen size is bogus. expect trouble
24/08/11 22:30:20 WARN Utils: Your hostname, LAPTOP-GTOHMK9K resolves to a loopback address: 127.0.1.1; using 172.24.196.18 instead (on interface eth0)
24/08/11 22:30:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/11 22:30:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
data_frame.show()

# from pyspark.sql import Row

# def shorten_names(row):
#     row_dict: dict = row.asDict()
#     cur_name = row_dict['product_name']
#     if len(cur_name) > 50:
#         word_list = cur_name.split(' ')
#         name = ' '.join([word.title() for word in word_list if word.isalpha()][:3])
#         row_dict.update({'product_name': name[:50]})
    
#     return Row(**row_dict)

# df_test = data_frame.rdd.map(shorten_names).toDF()
# df_test.show()

+--------------------+----------------+-----------------+----------+--------------------+-------------------+----------------+---+------+----------+--------------------+----------------+----------------------+----------------+-------------------+
|            order_id|     customer_id|    customer_name|product_id|        product_name|   product_category|    payment_type|qty| price|  datetime|             country|            city|ecommerce_website_name|  payment_txn_id|payment_txn_success|
+--------------------+----------------+-----------------+----------+--------------------+-------------------+----------------+---+------+----------+--------------------+----------------+----------------------+----------------+-------------------+
|2NEBnZETy1FdFpKJf...|wEwuo1GDdnbC8ju5|   STEPHEN THOMAS|     14315| Esr Boost Kickstand|tv, audio & cameras|             UPI|  3| 25.19|2022-05-19|       United States|       Blackhawk|       www.wayfair.com|XXFqXfizoOtsQ0S1|                  Y|
|cs4mesS4QiK

In [5]:
data_frame.createOrReplaceTempView('orders')

['Esr Boost Kickstand Case For Samsung Galaxy S23 Ultra 3 Stand Drop Supports', 'Rts Hdmi Cable 20 Meter Version Supports Full 4K 60Hz Audio', 'Mtr Mt901 Wireless Bluetooth Mini Card Support For Indoor Outdoor', 'Gadgetsden Belly Button Navel Piercing Ring Silver Color Heraldic Design Flower Opal Cubic Body Jewelry Barbells', 'Sg Musical 1 Set Of 4 Pcs Guitar String Guitar Accessories Bass Guitar Parts 4 Steel String Guitar Accessoriess Rr06', 'Crocs Classic Platform Glitter Flip', 'Flexnest Flexdubs True Wireless In Ear Earbuds With 40Db Anc Active Noise 35 Hour', 'Neewer 200Mm Professional Rail Nodal Slide Metal Quick Release Clamp For Camera With Arca Swiss Compatible', 'Puma Fabian Puma Yellow Running Shoes', 'Ribrona Digital Printed Poly Georgette Semi Stitched Lehenga Choli And Dupatta Set', 'Youbella Jewellery Gold Plated Charm Bracelet For Girls And Women', 'Barmunda Gems Ratti Certified Emerald Panna Panchdhatu Adjustable Rashi Ratan Ring For Astrological Purpose', 'Nike Men S

In [39]:



def get_popularity_over_year():
    query_split_datetime = 'CONCAT(YEAR(datetime), \'_Q\',QUARTER(datetime)) AS year_quarter'
    
    #Pivot to get popularity of each product over each quarter
    query_quarters = f'SELECT {query_split_datetime} FROM orders'
    query_unique_quarters = f'SELECT DISTINCT year_quarter FROM ({query_quarters}) ORDER BY year_quarter'
    quarters_df = context.sql(query_unique_quarters)
    quarter_list = quarters_df.collect()
    quarter_list = [row['year_quarter'] for row in quarter_list]

    query_pivot = '('
    for index, quarter in enumerate(quarter_list):
        if index == len(quarter_list) - 1:
            query_pivot += f'\'{quarter}\')'
        else:
            query_pivot += f'\'{quarter}\', '

    query_popularity_over_year_subquery = f'SELECT {query_split_datetime}, product_id, product_name, qty FROM orders'
    query_popularity_over_year = f'SELECT * FROM ({query_popularity_over_year_subquery}) PIVOT (SUM(qty) AS popularity FOR year_quarter IN {query_pivot})'
    
    pop_year_df = context.sql(query_popularity_over_year)
    pop_year_df = pop_year_df.na.fill(0)

    #query total popularity of each product over the year:
    query_total_popularity = f'SELECT product_id, product_name, SUM(qty) AS total_popularity FROM orders GROUP BY product_id, product_name'
    tot_pop_year_df = context.sql(query_total_popularity)
    #get top 10 most popular products:
    pop_year_df = pop_year_df.join(tot_pop_year_df,['product_id', 'product_name']).sort('total_popularity', ascending=False).limit(5)
    pop_year_df.show()
    
    # for index, quarter in enumerate(quarter_list):
    #     pop_year_df.sort(quarter, ascending=False).select(['product_id', 'product_name', quarter]).show(10)

    
    # pop_year_df.show(1000)
    return pop_year_df

def get_popularity_over_country():
    #get top 5 most popular products in each country with the total popularity of each product:
    query_top_products = '''
    SELECT country, product_id, product_name, SUM(qty) AS popularity
    FROM orders
    GROUP BY country, product_id, product_name
    ORDER BY country, popularity DESC
    '''
    pop_countries_df = context.sql(query_top_products)
    pop_countries_df.createOrReplaceTempView('pop_countries')
    query_top_products_per_country = '''
    SELECT country, product_id, product_name, popularity
    FROM (
        SELECT country, product_id, product_name, popularity,
               ROW_NUMBER() OVER (PARTITION BY country ORDER BY popularity DESC) AS rank
        FROM pop_countries
    ) tmp
    WHERE rank <= 5
    '''

    top_products_per_country_df = context.sql(query_top_products_per_country)
    # top_products_per_country_df.show()
    
    query_top_selling_countries = '''
    SELECT country, SUM(qty) AS total_popularity 
    FROM orders 
    GROUP BY country 
    ORDER BY total_popularity DESC 
    LIMIT 5
    '''

    pop_countries_df = context.sql(query_top_selling_countries)
    # pop_countries_df.show()

    top_selling_countries = pop_countries_df.select('country').collect()
    top_selling_countries = [row['country'] for row in top_selling_countries]

    pop_country_df = top_products_per_country_df.filter(top_products_per_country_df['country'].isin(top_selling_countries)).sort('country')
    pop_country_df.show(100)

    return pop_country_df

pop_year_df = get_popularity_over_year()
pop_country_df = get_popularity_over_country()

+----------+--------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+----------------+
|product_id|        product_name|2022_Q1|2022_Q2|2022_Q3|2022_Q4|2023_Q1|2023_Q2|2023_Q3|2023_Q4|2024_Q1|2024_Q2|2024_Q3|total_popularity|
+----------+--------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+----------------+
|    294020|      Jeans For Mens|      7|     20|     28|     42|     27|     31|     17|     23|      6|     11|      0|             212|
|     56727|Agaro Facial Steamer|      7|     22|     19|     19|     50|      2|     25|     34|     13|      7|      6|             204|
|    156025|Accessher Gold Pl...|     16|     11|     18|     21|      5|      8|     20|     21|     27|     35|     17|             199|
|    121492|Herbalife Afresh ...|      7|     29|     29|     24|     11|     13|     14|     40|     14|      7|     10|             198|
|    326224|       Aones Pa

In [40]:

# for jupyter
path1 = path + "/data/miguel/popularity_over_year.csv"
path2 = path + "/data/miguel/popularity_over_country.csv"
pop_year_df.toPandas().to_csv(str(path1), index=False)
pop_country_df.toPandas().to_csv(str(path2), index=False)